In [1]:
import os
from matplotlib import pyplot as plt
from web3 import Web3
from dotenv import load_dotenv
import math

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


WETH/USDC UniswapV3Pool - Friday 4th August 2023

In [2]:
sqrtPriceX96 = 1851710337453724344833367449611827

The price given in `slot0` is calculated as follows: $sqrtPriceX96 = \sqrt{price}*2^{96}$

Given that we want to calculate the `price` we have to perform several calculations.

1. Divide both sides by $2^{96}$

${sqrtPriceX96\over{2^{96}}} = \sqrt{price}$

2. Square both sides

${sqrtPriceX96\over{2^{96}}}^2 = price$

In [3]:
Q96 = 2 ** 96
price = (sqrtPriceX96 / Q96) ** 2
print(price)

546244320.767212


`WETH` has `18` decimals and `USDC` has `6` decimals. We have to adjust the price appropriately.

In [4]:
price = price / ((10 ** 18) / (10 ** 6))

The `price` calculated this way represents the value of `USDC` in `WETH`. Most exchanges represent it the other way around. How much `USDC` does 1 `WETH` cost.

In [5]:
print(price)

0.000546244320767212


We have to calculate the reverse: ${WETH\over{USDC}}^{-1} = {USDC\over{WETH}}$

In [6]:
price = price ** -1
print(price)

1830.6826487376165


Given the upper and lower ticks: 205419 and 202542 lets calculate the price.

In [7]:
tickUpper = 205419
tickLower = 202542

def tick_to_price(tick: int, token0_decimals: int, token1_decimals: int) -> float:
    return 1 / ((1.0001 ** tick) / ((10 ** token1_decimals) / (10 ** token0_decimals)))

# Price 0 is the price of USDC in WETH
price0 = (1.0001 ** tickUpper) / (10 ** 12)

# Price 1 is the price of WETH in USDC
price1 = 1/price0

assert price1 == tick_to_price(tickUpper, 6, 18), "By hand calc should be equal the function"
print(price1)
print(tick_to_price(tickLower, 6,18))

1200.0855710750832
1600.1197647925037


In [9]:
# Load environment variables from .env file
load_dotenv()
# Anvil forking mainnet
MAINNET_RPC_URL = os.environ.get('LOCALHOST_RPC_URL')
web3 = Web3(Web3.HTTPProvider(MAINNET_RPC_URL))